### **Proyecto 1 - Ingesta, Transformación, Carga y Reportería**

#### **Arquitectura del Proyecto**

<center><img src="https://i.postimg.cc/PJBMKtgL/db127.png"></center>

### **Paso 1 - Integración con ADLS**
_____

1. Pasos a seguir para realizar la integración con ADLS

2. Codificar el montaje (mount) con ADLS

### **Paso 2 - Data Ingestion**
_____

1. Ingesta manual de datos desde API hacia el contenedor **raw** en ADLS

2. Pasos a seguir en nuestra Ingesta de archivos desde el contenedor **raw** hacia el contenedor **ingested** (ambos en ADLS)

<center><img src="https://images2.imgbox.com/a1/9f/t0LTa2k6_o.png"></center> <!--db61-->

### Parte A

##### **Paso 2.1 - Ingesta del archivo "circuits.csv"**

1. Leer el archivo

2. Seleccionar solo las columnas que necesitamos

3. Cambiar el nombre de ciertas columnas

4. Añadir la fecha de ingestión al dataframe

5. Escribir datos en el contenedor **processed** del ADLS como **parquet**

<center><img src="https://i.postimg.cc/B6dF4Thd/db58.png"></center>

##### **Paso 2.2 - Ingesta del archivo "races.csv"**

1. Leer el archivo

2. Añadir las columnas "ingestion_date" y "race_timestamp"

3. Seleccionar sólo las columnas necesarias y renombrarlas como corresponda

4. Escribir datos en el contenedor **processed** del ADLS como **parquet**

<center><img src="https://i.postimg.cc/PqNskYvb/db59.png"></center>

##### **Paso 2.3 - Ingesta del archivo "constructors.json"**

1. Leer el archivo

2. Eliminar las columnas no deseadas

3. Cambiar el nombre de las columnas y añadir "ingestion date"

4. Escribir datos en el contenedor **processed** del ADLS como **parquet**

<center><img src="https://images2.imgbox.com/44/b5/oUpevSA8_o.png"></center> <!--db62-->

##### **Paso 2.4 - Ingesta del archivo "drivers.json"**

1. Leer el archivo

2. Renombrar columnas y añadir nuevas columnas

3. Eliminar las columnas no deseadas

4. Escribir datos en el contenedor **processed** del ADLS como **parquet**

<center><img src="https://images2.imgbox.com/e4/f7/eGqClVvp_o.png"></center> <!--db63-->

##### **Paso 2.5 - Ingesta del archivo "results.json"**

1. Leer el archivo

2. Renombrar columnas y añadir nuevas columnas

3. Eliminar las columnas no deseadas

4. Escribir datos en el contenedor **processed** del ADLS como **parquet**

<center><img src="https://images2.imgbox.com/bd/ee/T5xzXTnY_o.png"></center> <!--db64-->

##### **Paso 2.6 - Ingesta del archivo "pit_stops.json"**

1. Leer el archivo

2. Renombrar columnas y añadir nuevas columnas

3. Escribir datos en el contenedor **processed** del ADLS como **parquet**

<center><img src="https://i.postimg.cc/dV8W94f3/db65.png"></center>

##### **Paso 2.7 - Ingesta de los archivos "lap_times_split.csv"**

1. Leer el directorio **lap_times** el cual contiene multiples archivos CSV

2. Renombrar columnas y añadir nuevas columnas

3. Escribir datos en el contenedor **processed** del ADLS como **parquet**

<center><img src="https://i.postimg.cc/ZntjGQ01/db66.png"></center>

##### **Paso 2.8 - Ingesta de los archivos "qualifying_split.json"**

1. Leer el directorio **qualifying** el cual contiene multiples archivos Multi Line JSON

2. Renombrar columnas y añadir nuevas columnas

3. Escribir datos en el contenedor **processed** del ADLS como **parquet**

<center><img src="https://i.postimg.cc/pV94qxzr/db67.png"></center>

### Parte B

##### **Paso 2.9 - Databricks Workflows**

**Paso 2.9.1 - En primer lugar, veremos cómo incluir un notebook en otro notebook con parámetros de configuración y funciones comunes**

En cualquier lenguaje de programación, es importante escribir código reutilizable para que un proyecto sea más manejable y Spark no es una excepción. Spark nos permite reutilizar un notebook en otro notebook usando el comando mágico **%run**. Y esto ya lo hemos visto anteriormente, pero queremos aplicarlo a nuestro proyecto para que nuestra comprensión sea más concreta.

**Paso 2.9.2 - Definir parámetros en un notebook**

Vamos a ver cómo pasar parámetros a los notebooks de Databricks. En el punto 2.9.1 establecimos que el comando **%run** nos daba la capacidad de reutilizar código. Por ejemplo, si estás procesando datos de dos fuentes diferentes con las mismas características, en lugar de escribir dos notebooks diferentes, puedes escribir un notebook y enviar el nombre de la fuente de datos como parámetro. En nuestro proyecto, vamos a suponer que podemos obtener los datos de la "API eargast", como hemos comentado antes, o de la página web oficial de la Fórmula Uno directamente. Queremos procesar ambos conjuntos de datos utilizando los mismos notebooks, pero queremos almacenar el nombre de la fuente de datos junto a los datos para poder identificar el origen de los datos para cualquier propósito explicativo. En este caso, podemos parametrizar todos nuestros notebooks con el nombre de la fuente de datos como parámetro y enviar el valor en tiempo de ejecución. Los "widgets" de Databricks nos ayudan a hacerlo.

**Paso 2.9.3 - Invocar un notebook desde otro y encadenarlos y pasarles también los parámetros que hemos definido antes**

Ahora que tenemos ocho notebooks diferentes y que hasta ahora los hemos estado ejecutando a mano, puede que te estés preguntando cómo se ejecutan en escenarios de producción. Databricks nos brinda jobs que puedes programar para que se ejecuten a una hora específica, a un intervalo regular, lo que está muy bien, pero carece de muchas funcionalidades que normalmente obtendrías con cualquier tipo de herramientas de programación. Por ejemplo, no puedes crear dependencias entre jobs. Para solucionar eso, Databricks nos brinda una utilidad, llamada **Notebook Utility**, que te permite crear notebook workflow. Dicho esto, si estás en un Azure, no es probable que estés usando Databricks para crear workflows y programar jobs (scheduling jobs) en producción, debido a su funcionalidad inferior. Azure nos brinda una herramienta de workflow mucho mejor llamada **Azure Data Factory**, y eso es lo que utilizaremos. También me gustaría llevarte a través del notebook workflow y Databricks jobs para que puedas crear jobs rápidos para testear workloads sin tener que usar Azure Data Factory. 

**Paso 2.9.4 - Crear databricks jobs para ejecutar notebooks a intervalos regulares y bajo demanda**

Al final de este proceso, se espera lograr una buena comprensión de las capacidades que ofrece Databricks para crear notebook workflows y ejecutarlos desde databricks jobs.

### Parte C

##### **Paso 2.10 - Crear tablas con Databricks SQL**

**Paso 2.10.1 - Crearemos EXTERNAL TABLES para todos los archivos que se encuentren en el directorio "raw" del ADLS** 

La base de datos **f1_raw** se creará por defecto en la ruta **/user/hive/warehouse/f1_raw.db**

1. Una **external table** que haga referencia al archivo **circuits.csv** ubicada en la ruta **/mnt/formula1dl/raw/circuits.csv**

2. Una **external table** que haga referencia al archivo **races.csv** ubicada en la ruta **/mnt/formula1dl/raw/races.csv**

3. Una **external table** que haga referencia al archivo **constructors.json** ubicada en la ruta **/mnt/formula1dl/raw/constructors.json**

4. Una **external table** que haga referencia al archivo **drivers.json** ubicada en la ruta **/mnt/formula1dl/raw/drivers.json**

5. Una **external table** que haga referencia al archivo **results.json** ubicada en la ruta **/mnt/formula1dl/raw/results.json**

6. Una **external table** que haga referencia al archivo **pit_stops.json** ubicada en la ruta **/mnt/formula1dl/raw/pit_stops.json**

7. Una **external table** que haga referencia al directorio **lap_times** ubicada en la ruta **/mnt/formula1dl/raw/lap_times**

8. Una **external table** que haga referencia al directorio **qualifying** ubicada en la ruta **/mnt/formula1dl/raw/qualifying**


**Paso 2.10.2 - Crearemos MANAGED TABLES para todos los archivos ingestados y modificados**

La base de datos **f1_processed** se creará en la ruta **/mnt/formula1dl/processed**

1. Modificaremos la ruta de escritura de todos los notebooks, para escribir los datos ingestados en su tabla correspondiente

2. Debemos eliminar el directorio y archivo parquet creado previamente para todos los archivos, dado que, al guardarlo como tabla creará el directorio y archivo y además creará la tabla

2. Utilizaremos **lenguaje python** para crear las tablas, por lo que, por defecto serán **MANAGED TABLES**

### **Paso 3 - Data Transformation**
_____

1. Modificar nombres de columnas que no realizamos en la etapa de **Data Ingestion**

2. Utilizar las funciones **Filter** y **Join** para transformar los datos
```
    Utilizaremos la función "Join" sobre estos 5 dataframes
```    
<center><img src="https://i.postimg.cc/tC7fWR4B/db85.png"></center>

```
    Utilizaremos los siguientes campos
```
<center><img src="https://i.postimg.cc/7YRzX2FJ/db86.png"></center>

3. Utilizar las funciones **GroupBy** y **Agg** para transformar los datos

4. Utilizar las **Window functions** para transformar los datos

### Parte A

##### **Paso 3.1 - Utilizar las funciones "Filter" y "Join" para transformar los datos y asi obtener datos organizados que nos muestren los resultados de las carreras**

##### **Paso 3.2 - Utilizar las funciones "GroupBy/Agg" y "Window Functions" para transformar los datos y asi obtener datos organizados que nos muestren un ranking según el piloto**

##### **Paso 3.3 - Utilizar las funciones "GroupBy/Agg" y "Window Functions" para transformar los datos y asi obtener datos organizados que nos muestren un ranking según el team**

### Parte B

##### **Paso 3.4 - Crear tablas con Databricks SQL**

**Crearemos MANAGED TABLES para los tres archivos obtenidos en la transformación** 

La base de datos **f1_presentation** se creará en la ruta **/mnt/formula1dl/presentation**

1. Modificaremos la ruta de escritura de todos los notebooks, para escribir los datos transformados en su tabla correspondiente

2. Debemos eliminar el directorio y archivo parquet creado previamente para todos los archivos, dado que, al guardarlo como tabla creará el directorio y archivo y además creará la tabla

2. Utilizaremos **lenguaje python** para crear las tablas, por lo que, por defecto serán **MANAGED TABLES**

### **Paso 4 - Análisis**
_____

1. Crear una tabla de la cual nos apoyaremos para realizar nuestro análisis

2. Realizar un análisis de desempeño por piloto

3. Realizar un análisis de desempeño por equipo

### **Paso 5 - Patrón de diseño de carga de datos**
_____

1. Vamos a suponer que los datos nos llegarán de forma diaria:

    *   El dia 21 de Marzo de 2021 nos llegará toda la historia de carreras hasta la **raceid** igual a **1047**
    *   El dia 28 de Marzo de 2021 nos llegará solo la carrera de **raceid** igual a **1052**, que se realizó ese dia
    *   El dia 18 de Abril de 2021 nos llegará solo la carrera de **raceid** igual a **1053**, que se realizó ese dia
    *   Las carreras con **raceid** igual a **1049**, **1050** no se realizaron. La carrera de **raceid** igual a **1051** se postergó  
    *   El próximo archivo nos llegará el día 2 de Mayo de 2021 y corresponde a la **raceid** igual a **1054**

<center><img src="https://i.postimg.cc/J7Bc7Pp7/db138.png"></center>

2. Vamos a analizar la data de los archivos que se encuentran en los tres directorios:

    *   Primero, indicar que todos los dias que se haga una carrera nos llegará el archivo. Este será un directorio que llevará la fecha
        de la carrera por nombre y contendrá 8 archivos:

<center><img src="https://i.postimg.cc/pX507K94/db140.png"></center>    

3. Los archivos **circuits**, **races**, **constructors** y **drivers** son iguales para todos, es decir, la data que se encuentra en esos 4 archivos se repite en los 3 directorios **2021-03-21**, **2021-03-28** y **2021-04-18**. 

4. Los archivos **results**, **pit_stops** y los directorios con archivos **lap_times** y **qualifying**, solo traeran data correspondiente a
la carrera en cuestión. Por ejemplo, para la fecha **2021-03-21** el archivo **results** traerá data correspondiente a toda la historia de carreras hasta la **raceid** igual a **1047**. No así para el archivo **2021-03-28** para el cual traerá solo data de la carrera realizada 
dicho dia. Lo mismo para el dia **2021-04-18**

5. Podemos ver en el archivo **races** las carreras programadas desde el 29 de Marzo de 2009 hasta el 12 de Diciembre de 2021. Ahi podemos verificar las fechas en que irán realizando las proximas carreras. También comentar que la carrera de **raceid** igual a **1047** se realizó el 13 de Diciembre de 2020. Posteriormente, se comenzó a utilizar Azure y Databricks y los datos desde la primera carrera hasta la realizada en esa fecha, fue enviada el dia 21 de Marzo de 2021. A partir de la carrera de **raceid** igual a **1052** se fue enviando la data el dia que se realizó la carrera.

<center><img src="https://i.postimg.cc/NfjNs12d/db139.png"></center>

6. Antes de crear un patrón de diseño para la carga de nuestros datos, teniamos el siguiente patrón de carga donde cada vez que llega un directorio nuevo cada dia VOLVEMOS A CARGAR TODO y SOBRESCRIBIMOS LO ANTERIOR:

Nos llega el directorio **2021-03-21** y lo procesamos
<center><img src="https://i.postimg.cc/fRjVTgtd/db141.png"></center>

Ya tenemos en nuestro repositorio **raw** el directorio **2021-03-21** y ahora nos llegó el directorio **2021-03-28**. Procesamos ambos, sobrescribiendo todo lo realizado en el proceso anterior, es decir, lo realizado para el directorio **2021-03-21**
<center><img src="https://i.postimg.cc/GpX9v8jh/db142.png"></center>

Ya tenemos en nuestro repositorio **raw** el directorio **2021-03-21** y el directorio **2021-03-28** y ahora nos llegó el directorio **2021-04-18**. Procesamos los tres, sobrescribiendo todo lo realizado en el proceso anterior, es decir, lo realizado para el directorio **2021-03-21** y para el directorio **2021-03-28**
<center><img src="https://i.postimg.cc/mDhkjLd3/db143.png"></center>

En resúmen, siempre volvemos a procesar TODO y NO SÓLO LOS DATOS NUEVOS QUE HAN LLEGADO. 

##### **Paso 5.1 - Eliminar las Bases de datos y tablas de "f1_proccesed" y "f1_presentation" y volver a crearlas**

1. Comenzaremos vaciando el contenedor **raw** y cargando los directorios de los 3 dias, **2021-03-21**, **2021-03-28** y **2021-04-18**. La ruta para cada directorio vendria a ser de la siguiente forma:

    *   **/mnt/formula1dl/raw/2021-03-21**
    *   **/mnt/formula1dl/raw/2021-03-28**
    *   **/mnt/formula1dl/raw/2021-04-18**

<center><img src="https://i.postimg.cc/2jXCHXT7/db144.png"></center>

2. Eliminar las Bases de datos y tablas de **f1_proccesed** y **f1_presentation** y volver a crearlas

##### **Paso 5.2 - Método 1 de carga incremental utilizando el parámetro **p-file_date** del notebook**

1. Carga única de los archivos que son iguales en todos los dias: **circuits**, **races**, **constructors** y **drivers**

2. Modificar la **ruta de ingestión** utilizando el parámetro **p_file_date**

3. Modificar el **modo** de escritura, cambiamos de **overwrite** a **append**

3. Comenzar una carga incremental utilizando el parámetro **p-file_date** del notebook para los archivos **results** y **pit_stops** y los directorios con archivos **lap_times** y **qualifying**

##### **Paso 5.2 - Método 2 de carga incremental utilizando particiones**

##### **Paso 5.2 - Método 3 de carga incremental sobrescribiendo particiones**

##### **Paso 5.3 - Crear funciones y hacer más eficiente el código para el Método 3 de carga incremental**

##### **Paso 5.4 - Método de carga incremental para las transformaciones**

1. Método de carga incremental para la transformación **race_results**. Esta transformación la encontramos en el Paso 3 --> Paso B --> Paso 3.4.1

2. Método de carga incremental para la transformación **driver_standings**. Esta transformación la encontramos en el Paso 3 --> Paso B --> Paso 3.4.2

3. Método de carga incremental para la transformación **constructor_standings**. Esta transformación la encontramos en el Paso 3 --> Paso B --> Paso 3.4.3

### **Paso 6 - Crear una arquitectura utilizando Delta tables**
_____

##### **Paso 6.1 - Eliminar las Bases de datos y tablas de "f1_proccesed" y "f1_presentation" y volver a crearlas**

1. Comenzaremos vaciando el contenedor **raw** y cargando los directorios de los 3 dias, **2021-03-21**, **2021-03-28** y **2021-04-18**. La ruta para cada directorio vendria a ser de la siguiente forma:

    *   **/mnt/formula1dl/raw/2021-03-21**
    *   **/mnt/formula1dl/raw/2021-03-28**
    *   **/mnt/formula1dl/raw/2021-04-18**

<center><img src="https://i.postimg.cc/2jXCHXT7/db144.png"></center>

2. Eliminar las Bases de datos y tablas de **f1_proccesed** y **f1_presentation** y volver a crearlas

3. Solo utilizaremos las bases de datos **f1_proccesed** y **f1_presentation** y sus tablas. No utilizaremos la base de datos **f1_raw** ni sus tablas

##### **Paso 6.2 - Carga única de los archivos que son iguales en todos los dias:** **circuits**, **races**, **constructors** y **drivers**

- Reutilización del desarrollo de los archivos:

    *   **Paso 5.2.1**
    *   **Paso 5.2.2**
    *   **Paso 5.2.3**
    *   **Paso 5.2.4**
```
```
- Modificaremos el formato a **delta** (o también podriamos no indicarlo y aun asi creariamos una **tabla delta**. Si indicamos que su formato corresponde a **parquet** estariamos creando una **tabla parquet** y no una tabla delta) al momento de crear la tabla.         

##### **Paso 6.3 - Método de carga incremental para la ingestión**

- Reutilización del desarrollo de los archivos:

    *   **Paso 5.2.5**
    *   **Paso 5.2.6**
    *   **Paso 5.2.7**
    *   **Paso 5.2.8**
```
```
- Modificaremos nuestro método de carga incremental. Dado que estamos utilizando **tablas delta** tenemos la opción de utilizar un **MERGE** para ir insertando y/o actualizando nuestros datos

##### **Paso 6.4 - Método de carga incremental para las transformaciones**

- Reutilización del desarrollo de los archivos:

    *   **Paso 5.4.1**
    *   **Paso 5.4.2**
    *   **Paso 5.4.3**
```
```
- Modificaremos nuestro método de carga incremental. Dado que estamos utilizando **tablas delta** tenemos la opción de utilizar un **MERGE** para ir insertando y/o actualizando nuestros datos

##### **Paso 6.5 - Método de carga incremental para tabla de análisis de resultados**

### **Paso 7 - Orquestación utilizando Azure Data Factory**
_____